In [59]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [60]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [61]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [62]:
#Load the data and extract features for each sound file Actor_*/*.wav
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:/Users/Vaibhav Singh Takuli/OneDrive/Desktop/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [63]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)
x_train

array([[-5.22061890e+02,  3.50668907e+01,  3.75342965e+00, ...,
         1.65243153e-04,  1.04321596e-04,  6.55571566e-05],
       [-6.41227722e+02,  4.49487762e+01, -1.85174108e+00, ...,
         3.89261913e-05,  3.05255380e-05,  2.94166657e-05],
       [-6.50705750e+02,  5.30211639e+01, -4.92040443e+00, ...,
         4.75216802e-05,  3.46632514e-05,  1.62844426e-05],
       ...,
       [-5.50096191e+02,  1.70297680e+01, -1.14575644e+01, ...,
         1.51764631e-04,  1.16828531e-04,  8.47479314e-05],
       [-5.55357605e+02,  4.71569710e+01,  1.10750732e+01, ...,
         1.61086471e-04,  1.04962463e-04,  6.52811723e-05],
       [-5.04816345e+02,  3.53618660e+01, -1.43495789e+01, ...,
         6.08151546e-04,  5.55269769e-04,  4.47782222e-04]])

In [64]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [65]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [66]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [67]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [68]:
#Predict for the test set
y_pred=model.predict(x_test)
y_pred

array(['happy', 'fearful', 'happy', 'happy', 'fearful', 'fearful',
       'disgust', 'disgust', 'calm', 'fearful', 'happy', 'fearful',
       'fearful', 'happy', 'disgust', 'happy', 'calm', 'disgust', 'happy',
       'calm', 'disgust', 'disgust', 'disgust', 'calm', 'happy', 'happy',
       'fearful', 'happy', 'fearful', 'fearful', 'happy', 'disgust',
       'happy', 'fearful', 'happy', 'calm', 'fearful', 'fearful', 'calm',
       'disgust', 'happy', 'disgust', 'fearful', 'calm', 'fearful',
       'calm', 'disgust', 'calm', 'happy', 'happy', 'fearful', 'fearful',
       'fearful', 'happy', 'happy', 'fearful', 'disgust', 'happy', 'calm',
       'calm', 'disgust', 'calm', 'happy', 'happy', 'disgust', 'calm',
       'fearful', 'disgust', 'fearful', 'happy', 'fearful', 'fearful',
       'fearful', 'fearful', 'fearful', 'disgust', 'fearful', 'happy',
       'fearful', 'fearful', 'calm', 'calm', 'fearful', 'calm', 'disgust',
       'fearful', 'disgust', 'fearful', 'happy', 'fearful', 'disgust

In [69]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 61.46%


In [71]:
from sklearn.metrics import accuracy_score, f1_score
f1_score(y_test, y_pred,average=None)

array([0.62365591, 0.6097561 , 0.58181818, 0.64646465])

In [72]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,calm,fearful
2,happy,happy
3,happy,happy
4,disgust,fearful
5,calm,fearful
6,happy,disgust
7,happy,disgust
8,disgust,calm
9,happy,fearful


In [73]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [74]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("C:/Users/Vaibhav Singh Takuli/OneDrive/Desktop/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [75]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450770e-01,
         1.43299656e+01,  3.33636999e+00, -2.54071975e+00,
        -4.05790901e+00, -1.07119999e+01, -7.29414034e+00,
         1.74018860e+00, -4.19064283e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78143024e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803221e+00, -5.68717718e-01,
        -6.47753382e+00, -1.24320555e+00, -2.80542493e+00,
        -5.43635798e+00, -4.46875274e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902497e-01, -1.65101981e+00,
        -5.55944800e-01, -3.41018438e+00, -2.24465513e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821304e+00,
        -5.54154336e-01, -3.96459842e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457895e+00, -2.03990722e+00,
        -3.62910867e+00,  6.33734703e-01,  6.48760557e-01,
         6.21744156e-01,  6.34553194e-01,  6.60345614e-01,
         6.60463989e-01,  7.00949013e-01,  7.31556237e-01,
         7.46921480e-01,  7.23420441e-01,  7.16119647e-0